In [1]:
import re
import matplotlib.pyplot as plt
import numpy as np


space_split = re.compile("( |\n)")#("(### Batch [0-9]+)")
time_find = re.compile("::CLINK:.+?\n")
cost_find = re.compile("COST =.+?\n")
mem_heap_find = re.compile("mem_heap_B=.+?\n")
mem_heap_extra_find = re.compile("mem_heap_extra_B=.+?\n")

In [2]:
def getPrintMethod(method):
    if method == "complete":
        method = "comp"
    elif method == "ward":
        method = "Ward"
    elif method == "avg":
        method = "avg-1"
    elif method == "avg32":
        method = "avg-1"
    elif method == "avgsq":
        method = "avg-2"
    elif method == "avgsq_matrix":
        method = "avg-2"
    elif method == "avgsq_matrix_range":
        method = "avg-2"
    elif method == "complete_matrix_range":
        method = "comp"
    elif method == "ward_matrix_range":
        method = "Ward"
    elif method == "avg_matrix_range":
        method = "avg-1"
    return method

data_file_sizes = {
    "2D_GaussianDisc_1K": 37829,
    "10D_UCI1_19K": 1560440,
    "2D_GaussianDisc_10K":379581,
    "2D_GaussianDisc_100K":3736129,
    "2D_GaussianDisc_3K":113798,
    "2D_UniformFill_1M":37771976,
    "2D_GaussianDisc_1M":37370550,
}

data_sizes = {
    "10D_UCI1_19K": 19020,
    "2D_GaussianDisc_1K":1000,
    "2D_GaussianDisc_3K":3000,
    "2D_GaussianDisc_10K":10000,
    "2D_GaussianDisc_100K":100000,
    "2D_UniformFill_1M":1000000,
    "2D_GaussianDisc_1M":1000000,
}

In [8]:
def get_times(method, dataset, worker, opt = False):
    impl="PC"
    run_split = re.compile("\+\+\+\+\+\+\+\+\+\+\+\+\+\+\+\n")#("(### Batch [0-9]+)")
#     if(method[0:7] == "single"):
#         run_split = re.compile("========= Boruvka =========\n")#("(### Batch [0-9]+)")
    num_runs = 3
    if(worker==1 or opt):
        num_runs = 1
    plotthread = worker # number of threads on the plot 
    if worker== 96:
        plotthread = 60
#     if worker==72:
#         plotthread = 60
    base_addr = "/Users/sy/Desktop/MIT/HAC/parChain/outputs/outputs_%s/" % method
    file = base_addr+"%s_%s_%sth.txt" % (method, dataset, worker)
    if opt:
        base_addr = "/Users/sy/Desktop/MIT/OPTICS/pbbs/linkage/outputs_opt/"
        file = base_addr+"%s_norange_%s_%sth.txt" % (method, dataset, worker)
    if "matrix" in method:
        base_addr = "/Users/sy/Desktop/MIT/OPTICS/pbbs/linkage/outputs_matrix/"
        file = base_addr+"%s_%s_%sth.txt" % (method, dataset, worker)
        impl="PC-matrix"
        if "range" in method:
            base_addr = "/Users/sy/Desktop/MIT/HAC/parChain/outputs/outputs_matrix_range/"
            file = base_addr+"%s_%s_%sth.txt" % (method, dataset, worker)
            impl="PC-matrix-r"
    f1 = open(file)  
    data1 = f1.read()
    runs = run_split.split(data1) #runs[1:1+num_runs] are the three runs
    times = []
    cost = float(cost_find.findall(runs[1])[0][7:])

    method = getPrintMethod(method)
    
    for i in range(num_runs):
        times.append(float(time_find.findall(runs[1+i]+"\n")[0][9:]))
        diff = abs(cost - float(cost_find.findall(runs[1+i])[0][7:]))
        if (diff > cost * 0.000001 or diff > 0.1):
            print(dataset, i, cost, float(cost_find.findall(runs[1+i])[0][7:]))
    if opt:
        return ("%s,%s,%s,%s,%s,PC-no-range,%s,%s \n" % (dataset.replace("_","-"), method,cost, min(times), worker,  max(times), plotthread))        
    return ("%s,%s,%s,%s,%s,%s,%s,%s \n" % (dataset.replace("_","-"), method,cost, min(times), worker,  impl, max(times), plotthread))

In [4]:
def processTimes(method, workers = [1, 12, 24, 36, 48, 96]):
    filename = "runtimeProcessed/%s_times_us.csv" %method
    file1 = open(filename, 'w')
    for dataset in [    "10D_UCI4_100K" ,\
        "10D_UCI1_19K" ,\
        "2D_GaussianDisc_10K",\
        "2D_UniformFill_1M",\
        "2D_GaussianDisc_1M",\
        "5D_UniformFill_1M",\
        "5D_GaussianDisc_1M",\
        "2D_UniformFill_10M",\
        "2D_GaussianDisc_10M",\
        "5D_UniformFill_10M",\
        "5D_GaussianDisc_10M",\
        "3D_GeoLife_24M",\
        "HT",\
        "CHEM", \
        "2D_GaussianDisc_100K",\
         "2D_GaussianDisc_5M"]:
        for worker in workers:
            try:
                file1.write(get_times(method, dataset, worker) )
            except:
                plotthread = worker # number of threads on the plot 
                if worker== 96:
                    plotthread = 60
                impl="PC"
                if "matrix" in method:
                    impl="PC-matrix"
                    if "range" in method:
                        impl = "PC-matrix-r"
                file1.write("%s, , , , %s, %s,,%s \n" % (dataset.replace("_","-"),  worker, impl,plotthread))
    file1.close()

In [ ]:
get_times(method, dataset, worker)

In [24]:
method = "avgsq_matrix_range"
processTimes(method)

In [268]:
method = "ward"
processTimes(method)

In [269]:
method = "avgsq"
processTimes(method)

In [12]:
method = "avg2"
processTimes(method, [96])

In [271]:
method = "avg128"
processTimes(method)

In [272]:
method = "avg256"
processTimes(method, [96])

In [273]:
method = "avg64"
processTimes(method, [96])

In [274]:
method = "avg16"
processTimes(method, [96])

In [275]:
method = "avg1"
processTimes(method, [96])

In [276]:
method = "avgsq_matrix"
processTimes(method)

In [280]:
def processTimesOpt():
    filename = "runtimeProcessed/norange_times_us.csv" 
    file1 = open(filename, 'w')
    for method in ["complete", "ward", "avg", "avgsq"]:
        for dataset in ["10D_UCI4_100K" , "2D_UniformFill_1M"]:
            try:
                file1.write(get_times(method, dataset, 96, True) )
            except:
                method_print = getPrintMethod(method)
                file1.write("%s,%s,,0,%s,PC-no-range,, \n" % (dataset.replace("_","-"),  method_print, worker))
    file1.close()

In [282]:
processTimesOpt()

In [27]:
def getMaxTotalMemo(method, dataset, worker, impl):
    filename = "./memo_outputs/%smemo_%s_%sth.out" %(method, dataset, worker)
    if(impl == "scipy" or impl == "sklearn"or impl == "fastprotein" or impl == "fastcluster"):
        filename = "./memo_outputs/%s_%smemo_%s_%sth.out" %(impl, method, dataset, worker)
    if(impl == "Jeon"):
        filename = "./memo_outputs/jeonmemo_%s_%sth.out" %( dataset, worker)
    if("matrix" in impl):
        filename = "./memo_outputs/%s_matrixmemo_%s_%sth.out" %(method, dataset, worker)
        if("range" in impl):
            filename = "./memo_outputs/%s_matrixrangememo_%s_%sth.out" %(method, dataset, worker)
    file1 = open(filename, 'r')
    data1 = file1.read()
    mem_heap_times = mem_heap_find.findall(data1)  
    mem_heap_extra_times = mem_heap_extra_find.findall(data1)    
    times= []
    for i in range(len(mem_heap_times)):
        mem_heap_time = int(mem_heap_times[i][11:-1])
        mem_heap_extra_time = int(mem_heap_extra_times[i][17:-1])  
        times.append(mem_heap_time+mem_heap_extra_time)
    method = getPrintMethod(method)
    size = data_file_sizes[dataset]/1e6
    return "%s,%s,%s,%s,%s,%s \n" %(dataset.replace("_","-"), method,  impl, max(times)/1e6 , size, data_sizes[dataset]) ## MB

def processMemo():
    filename = "runtimeProcessed/memory_us.csv" 
    file1 = open(filename, 'w')
    for impl in ["PC-matrix-r"]:# ["PC", "PC-matrix", "fastcluster", "scipy", "sklearn", "fastprotein", "Jeon"]:
        if(impl == "PC"):      
            for method in ["complete", "ward", "avg32", "avgsq"]:
                for dataset in ["2D_GaussianDisc_1K", "2D_GaussianDisc_3K" , "2D_GaussianDisc_10K"]:
                    try:
                        file1.write(getMaxTotalMemo(method, dataset, 96, impl))
                    except:
                        method_print = getPrintMethod(method)
                        size = data_file_sizes[dataset]/1e6
                        file1.write("%s,%s,PC,,%s,%s \n" % (dataset.replace("_","-"),  method_print, size, data_sizes[dataset]))
        if(impl == "PC-matrix"):      
            for method in ["complete", "ward", "avg", "avgsq"]:
                for dataset in ["2D_GaussianDisc_1K", "2D_GaussianDisc_3K" , "2D_GaussianDisc_10K"]:
                    try:
                        file1.write(getMaxTotalMemo(method, dataset, 96, impl))
                    except:
                        method_print = getPrintMethod(method)
                        size = data_file_sizes[dataset]/1e6
                        file1.write("%s,%s,PC-matrix,,%s,%s \n" % (dataset.replace("_","-"),  method_print, size, data_sizes[dataset]))
        if(impl == "PC-matrix-r"):      
            for method in ["complete", "ward", "avg", "avgsq"]:
                for dataset in ["2D_GaussianDisc_1K", "2D_GaussianDisc_3K" , "2D_GaussianDisc_10K"]:
                    try:
                        file1.write(getMaxTotalMemo(method, dataset, 96, impl))
                    except:
                        method_print = getPrintMethod(method)
                        size = data_file_sizes[dataset]/1e6
                        file1.write("%s,%s,PC-matrix-r,,%s,%s \n" % (dataset.replace("_","-"),  method_print, size, data_sizes[dataset]))
        elif(impl == "fastprotein"):      
            for method in ["complete",  "avg"]:
                for dataset in ["2D_GaussianDisc_1K", "2D_GaussianDisc_3K" , "2D_GaussianDisc_10K"]:
                    try:
                        file1.write(getMaxTotalMemo(method, dataset, 36, impl))
                    except:
                        method_print = getPrintMethod(method)
                        size = data_file_sizes[dataset]/1e6
                        file1.write("%s,%s,%s,,%s,%s \n" % (dataset.replace("_","-"),  method_print, impl, size, data_sizes[dataset]))
        elif(impl == "Jeon"):
            method = "avg"
            for dataset in ["2D_GaussianDisc_1K", "2D_GaussianDisc_3K" , "2D_GaussianDisc_10K"]:
                try:
                    file1.write(getMaxTotalMemo(method, dataset, 4, impl))
                except:
                    method_print = getPrintMethod(method)
                    size = data_file_sizes[dataset]/1e6
                    file1.write("%s,%s,jeon,,%s, \n" % (dataset.replace("_","-"),  method_print, size))
        elif(impl == "scipy" or impl == "sklearn" or impl == "fastcluster"):
            for method in ["complete", "ward", "avg", "avgsq"]:
                for dataset in ["2D_GaussianDisc_1K", "2D_GaussianDisc_3K" , "2D_GaussianDisc_10K"]:
#                     if(impl == "sklearn" and method=="avgsq" and dataset == "2D_GaussianDisc_100K" ):
#                         method_print = getPrintMethod(method)
#                         file1.write("%s,%s,%s,,%s,%s \n" % (dataset.replace("_","-"),  method_print, impl, size, data_sizes[dataset]))
#                         continue
                    try:
                        file1.write(getMaxTotalMemo(method, dataset, 1, impl))
                    except:
                        method_print = getPrintMethod(method)
                        size = data_file_sizes[dataset]/1e6
                        file1.write("%s,%s,%s,,%s,%s \n" % (dataset.replace("_","-"),  method_print, impl, size,data_sizes[dataset]))

    file1.close()

In [28]:
processMemo()

In [284]:
def getJeonTime(dataset, worker):
    filename = "./outputs_jeon/jeon_%s_%sth.txt" %(dataset, worker)
    file1 = open(filename, 'r')
    data1 = file1.read()
    runs = data1.split("\n\n")

    times= []
    rounds = 3
    if(worker == 1):
        rounds=1
    for i in range(rounds):
        time = float(runs[i+1].split("Total")[-1])
        times.append(time)
    # method = getPrintMethod(method)
    # size = data_file_sizes[dataset]/1e6
    plotthread = worker # number of threads on the plot 
    if worker== 96:
        plotthread = 60
    return ("%s,avg-1,,%s,%s,Jeon,%s,%s \n" % (dataset.replace("_","-") , min(times), worker,  max(times), plotthread))        
def processJeon(workers):
    filename = "runtimeProcessed/avg_times_jeon.csv" 
    file1 = open(filename, 'w')
    for dataset in ["10D_UCI1_19K", "2D_GaussianDisc_10K" , "10D_UCI4_100K"]:
        for worker in workers:
            try:
                file1.write(getJeonTime(dataset, worker))
            except:
                plotthread = worker # number of threads on the plot 
                if worker== 96:
                    plotthread = 60
                file1.write("%s,avg-1,,,%s,Jeon,,%s \n" % (dataset.replace("_","-") , worker,  plotthread))
    file1.close()


In [286]:
processJeon([1, 2, 4, 8, 12, 24, 36, 48, 96])

In [256]:
def processFastCluster():
    filename = "./fastprotein_outputs.txt"
    writefilename1 = "runtimeProcessed/avg_times_fastprotein.csv" 
    writefile1 = open(writefilename1, 'w')
    writefilename2 = "runtimeProcessed/complete_times_fastprotein.csv" 
    writefile2 = open(writefilename2, 'w')
    file1 = open(filename, 'r')
    data1 = file1.read()
    runs = data1.split("./fast_protein_cluster -i ~/datasets/")
    run = runs[1]
    for run in runs[1:]:
        dataset = run[:run.find(".pbbs")]

        method = "comp"
        if("--haverage" in run):
            method = "avg-1"
            workerind = run.find(" --haverage")

        else:
            workerind = run.find(" --hcomplete")
        worker = run[workerind-2:workerind]

        ioind1 = run.find("for reading...\n   ")
        ioind2 = run.find(" seconds elapsed for coords I/O")    
        iotime = float(run[ioind1+18:ioind2])
        totalind2 = run.find(" seconds total time ")
        totalind1 = run[:totalind2].rfind(" ")
        totaltime = float(run[totalind1:totalind2])

        plotthread = worker # number of threads on the plot 
        if worker== 96:
            plotthread = 60
        if method == "avg-1":
            writefile1.write("%s,%s,,%s,%s,fastprotein,,%s \n" % (dataset.replace("_","-") , method, totaltime-iotime, worker,  plotthread))
        else:
            writefile2.write("%s,%s,,%s,%s,fastprotein,,%s \n" % (dataset.replace("_","-") , method, totaltime-iotime, worker,  plotthread))

    writefile1.close()
    writefile2.close()

In [ ]:
processFastCluster()